# Water Quality CSV Transects joined by TIME/DATE 

In [1]:
import os
import pandas
from scripts import wqt_timestamp_match
from datetime import datetime

In [2]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# example shp
shp = os.path.join(wd, "scripts", "testfiles", "Arc_040413\\Arc_040413_GPS\\040413_PosnPnt.shp")

# example water quality
wq_file = os.path.join(wd, "scripts", "testfiles", "Arc_040413\\Arc_040413_WQ\\Arc_040413_wqt_cc.csv")


print(shp)
print(wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\scripts\testfiles\Arc_040413\Arc_040413_GPS\040413_PosnPnt.shp
C:\Users\Andy\Documents\arcproject-wq-processing\scripts\testfiles\Arc_040413\Arc_040413_WQ\Arc_040413_wqt_cc.csv


### Import CSV file from the Sonde as Pandas Dataframe

In [3]:
water_quality = wqt_timestamp_match.wq_from_csv(wq_file)
print(water_quality.head())

            Date_Time   Temp    pH SpCond   Sal DEP25  PAR RPAR TurbSC   CHL  \
1 2013-04-04 13:01:03  17.40  7.84    520  0.26  0.39  573  417   13.4  5.34   
2 2013-04-04 13:01:04  17.40  7.84    520  0.26  0.35  573  417   13.4  5.34   
3 2013-04-04 13:01:05  17.40  7.88    520  0.26  0.35  576  419   13.5  5.29   
4 2013-04-04 13:01:06  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   
5 2013-04-04 13:01:07  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   

     CHL.1              WQ_SOURCE  
1  0.05078  Arc_040413_wqt_cc.csv  
2  0.05078  Arc_040413_wqt_cc.csv  
3  0.05095  Arc_040413_wqt_cc.csv  
4  0.05073  Arc_040413_wqt_cc.csv  
5  0.05073  Arc_040413_wqt_cc.csv  


### Transect Shapefile 

In [4]:
shp_df = wqt_timestamp_match.shp2gpd(shp)

print(shp_df.head())

     GPS_Date    GPS_Time                                      geometry  \
0  2013-04-04  08:18:47am   POINT (-121.6759818490551 38.2332247673879)   
1  2013-04-04  08:18:48am  POINT (-121.6759860012121 38.23321961147579)   
2  2013-04-04  08:18:49am  POINT (-121.6759904209636 38.23321431496129)   
3  2013-04-04  08:18:50am  POINT (-121.6759940577057 38.23320903904457)   
4  2013-04-04  08:18:51am  POINT (-121.6759974280661 38.23320393778903)   

           GPS_SOURCE           Date_Time  
0  040413_PosnPnt.shp 2013-04-04 08:18:47  
1  040413_PosnPnt.shp 2013-04-04 08:18:48  
2  040413_PosnPnt.shp 2013-04-04 08:18:49  
3  040413_PosnPnt.shp 2013-04-04 08:18:50  
4  040413_PosnPnt.shp 2013-04-04 08:18:51  


In [5]:
# combine the GPS_Date and GPS_Time fields with the geopandas dataframe in order to create a field to join on 
date = shp_df['GPS_Date'][0]
time = shp_df['GPS_Time'][0]

print(date, time)

dt = date + 't' + time
print(dt)

date_object = datetime.strptime(dt, '%Y-%m-%dt%I:%M:%S%p')
print(date_object)

('2013-04-04', u'08:18:47am')
2013-04-04t08:18:47am
2013-04-04 08:18:47


### Join the transect geodataframe with the water quality dataframe

In [10]:
merged = wqt_timestamp_match.JoinByTimeStamp(water_quality, shp_df)
print(merged.head())

print(shp_df.shape)
print(water_quality.shape)
print(merged.shape)


#match = wqt_timestamp_match.splitunmatched(merged)
joined_data = merged
match = joined_data.dropna(axis='index')
no_geo = joined_data[joined_data["GPS_SOURCE"].isnull()]
no_wq = joined_data[joined_data["WQ_SOURCE"].isnull()]

print(match.shape)
print(no_geo.shape)
print(no_wq.shape)

#print('Dimensions - origninal WQ:  {},  Matches: {}'.format( water_quality.shape, matches.shape))

#print('Number of rows different: {}'.format(water_quality.shape[0] - matches.shape[0]))

#percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)

#print('Percent Match: {} %'.format(percent_match))

     GPS_Date    GPS_Time                                      geometry  \
0  2013-04-04  08:18:47am   POINT (-121.6759818490551 38.2332247673879)   
1  2013-04-04  08:18:48am  POINT (-121.6759860012121 38.23321961147579)   
2  2013-04-04  08:18:49am  POINT (-121.6759904209636 38.23321431496129)   
3  2013-04-04  08:18:50am  POINT (-121.6759940577057 38.23320903904457)   
4  2013-04-04  08:18:51am  POINT (-121.6759974280661 38.23320393778903)   

           GPS_SOURCE           Date_Time Temp   pH SpCond  Sal DEP25  PAR  \
0  040413_PosnPnt.shp 2013-04-04 08:18:47  NaN  NaN    NaN  NaN   NaN  NaN   
1  040413_PosnPnt.shp 2013-04-04 08:18:48  NaN  NaN    NaN  NaN   NaN  NaN   
2  040413_PosnPnt.shp 2013-04-04 08:18:49  NaN  NaN    NaN  NaN   NaN  NaN   
3  040413_PosnPnt.shp 2013-04-04 08:18:50  NaN  NaN    NaN  NaN   NaN  NaN   
4  040413_PosnPnt.shp 2013-04-04 08:18:51  NaN  NaN    NaN  NaN   NaN  NaN   

  RPAR TurbSC  CHL CHL.1 WQ_SOURCE  
0  NaN    NaN  NaN   NaN       NaN  
1  NaN

In [7]:
# adjust date time by an hour to account for uncorrected daylight savings time

from datetime import timedelta


def dstadjustment(df, offset_hours):
    df2 = df.copy() # make a copy of data so original is not overwritten
    dstshift = lambda x: x + timedelta(hours=offset_hours)
    df2['Date_Time'] = df2['Date_Time'].map(dstshift)
    return df2



#orignal = timestamp_match.JoinByTimeStamp(water_quality, shp_df)
#percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)


#plus1 = dstadjustment(water_quality, 1)

offsets= {}

for i in [-1,-0.5, 0, 0.5,1]:
    print("Time offset: {} hour".format(i))
    
    # offset water quality
    off = dstadjustment(water_quality, i)
    
    # try joining by timestamp
    matches = timestamp_match.JoinByTimeStamp(off, shp_df)[0]
    
    # report the percentage matched
    percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)
    print("{} %".format(percent_match))
    
    # add to dict
    offsets[i]=percent_match

print(offsets)
highest_percent_offset = max(offsets, key=offsets.get)
print(highest_percent_offset)


offset_df = timestamp_match.dstadjustment(water_quality, highest_percent_offset)
offset_df.head()

# join using time stamps w/ exact matchjoined_data = timestamp_match.JoinByTimeStamp(offset_df, pts)

Time offset: -1 hour


NameError: name 'timestamp_match' is not defined

## transect_join_timestamp - batch function to combine transect water quality with GPS

In [ ]:
joined_data = timestamp_match.transect_join_timestamp(wq_file, shp)
print(joined_data)

In [ ]:
# save to shapefile

timestamp_match.main(wq_file, shp, "Example.shp")

# ZOOP CHL W points

In [ ]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# shapefile of the Chl sampling locations
station_shp = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_GPS", "040413_ZoopChlW.shp")
# water quality vertical profile for a Chl sampling point
station_wq_file = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_WQ", "Arc_040413_wqp_ca1.csv")

print(station_shp)
print(station_wq_file)

In [ ]:
# let's process that water quality at the stationary point
station_water_quality = timestamp_match.wq_from_csv(station_wq_file)
print(station_water_quality.head())

In [ ]:
# let's get the shapefile for the stationary points collected that day.

station_shp_df = timestamp_match.shp2dataframe(station_shp)

print(station_shp_df.head())

In [ ]:
# combine the GPS_date and GPS time fields to match a timestamp to use as join field
timestamp_match.addCombinedDateTime(station_shp_df, "GPS_Date", "GPS_Time")
print(station_shp_df.head())

In [ ]:
# for each row in the water quality dataframe join the station in if it's with xx minutes. 
station_gps_datetimes = station_shp_df["Date_Time"].tolist()
print(station_gps_datetimes)

# for each row find the closest time stamp


import datetime

ex = station_water_quality["Date_Time"][10]
print("example: {}".format(ex))



row_time = ex
print("Test: {}".format(row_time))


def my_minimum(iterable):
    iterable = iter(iterable)
    minimum = iterable.next()
    for i in iterable:
        if i < minimum:
            minimum = i
    return minimum

print(my_minimum(station_gps_datetimes))


def find_closest_time(row_time, list_of_potential_matches):
    iterable = iter(list_of_potential_matches)
    minimum = iterable.next()
    for t = iterable

    for sample_time in list_of_potential_matches:
        diff = (row_time - sample_time)
        
        if diff < 1:
            closest_time = sample_time
            gap = diff
        print(gap)
            
    # check if closest time is 1 hour        
    if gap == datetime.timedelta(hours=1):
        return "No Match"
    else:
        return closest_time

print(find_closest_time(row_time, station_gps_datetimes))